<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [45]:
import pandas as pd

# буду использовать alchemy, так как предлагает jyputer notebook
from sqlalchemy import create_engine
from sqlalchemy import text

import requests
from bs4 import BeautifulSoup

import plotly.express as px

### Схема подключения к БД.
![](images/SQL_Alchemy.png)

*Здесь пул — это пул соединений к БД. Этот класс держит открытые соединения к БД и кэширует запросы.*

*Диалект — класс, специфицирует обращение к конкретной связке базы данных и её  API.*

*Движок(англ. engine) — класс содержащий объект и пула, и диалекта и совмещающий в себе их работу. Как я понял, выполнен по шаблону проектирования Фасад.*

*Метод connect() — возвращает новый объект ‘подключение’ (англ. connection object), и предоставляет сервис для выполнения SQL выражений и также контроля транзакций.*
*Подключение — это прокси-объект для фактического DB API объекта подключения.*

In [47]:
# the string form of the URL starts with dialect[+driver] in our case it is 'postgresql+psycopg2'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

### ERD базы данных проекта.
![](images/project_ed.png)

# Юнит 3. Предварительный анализ данных

Для знакомства  с таблицами будет составлен запрос, который получает
количество хранимых в каждой таблице базы данных строк.
<br><br>
Посредством него будет получено:
 - количество вакансий в базе данных (вакансии находятся в таблице vacancies) 
 - количество работодателей (таблица employers)
 - количество регионов (таблица areas)
 - количество сфер деятельности в базе (таблица industries)

In [48]:
# получаем количество строк в таблицах
query = f'''SELECT COUNT(*) 
            FROM public.vacancies
            UNION ALL
            SELECT COUNT(*) 
            FROM public.employers
            UNION ALL
            SELECT COUNT(*) 
            FROM public.areas
            UNION ALL
            SELECT COUNT(*) 
            FROM public.industries
         '''

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.fetchall()

In [49]:
print(f'Количество вакансий в БД: {answer[0][0]}')

Количество вакансий в БД: 49197


In [50]:
print(f'Количество работодателей в БД: {answer[1][0]}')

Количество работодателей в БД: 23501


In [51]:
print(f'Количество регионов охваченных БД: {answer[2][0]}')

Количество регионов охваченных БД: 1362


In [52]:
print(f'Количество сфер деятельности охваченных БД: {answer[3][0]}')

Количество сфер деятельности охваченных БД: 294


***

По предварительному обзору количества хранимой в таблицах информации можно сделать несколько **выводов**:
   - На момент составления базы на рынке каждый работадатель мог бы предложить чуть более двух вакансий, если распределить число вакансий равномерно по числу работадателей. Хотя очевидно, что чем крупнее компания тем больше вакансий она предлагает. 
   - Количество охваченных регионов, очень широко. Вероятно включены не только города и регионы России: так 
    как количество городов в России составляет 1112 штук на 2022 год, а регионов меньше 100. Это говорит о присутствии населённых пунктов других типов
    и/или других стран.
   - Количество сфер деятельности также очень разнообразно, возможно присутсвуют синонимичные названия.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [53]:
# ограничение количества выводимых строк
n = 20

query = f'''SELECT a.name,
                   COUNT(v.id) cnt  
            FROM public.vacancies v
            JOIN areas a ON v.area_id = a.id
            GROUP BY a.id
            ORDER BY cnt DESC
            LIMIT {n}
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df.style.hide(axis='index'))

name,cnt
Москва,5333
Санкт-Петербург,2851
Минск,2112
Новосибирск,2006
Алматы,1892
Екатеринбург,1698
Нижний Новгород,1670
Казань,1415
Краснодар,1301
Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [54]:
# получаем количество строк в таблицах
query = f'''SELECT COUNT(v.id)  
            FROM public.vacancies v
            WHERE v.salary_from IS NOT NULL OR
                  v.salary_to IS NOT NULL
         '''

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [55]:
# результат запроса
print(f'Количество вакансий где указана минимальная \
или максимальная ожидаемая зарплата: {answer[0]}')

Количество вакансий где указана минимальная или максимальная ожидаемая зарплата: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [56]:
query = f'''SELECT ROUND(AVG(v.salary_from), 0) avg_min_salary,
                   ROUND(AVG(v.salary_to), 0) avg_max_salary
            FROM public.vacancies v
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

display(df.style.hide(axis='index').format(precision=1))

avg_min_salary,avg_max_salary
71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [57]:
query = f'''SELECT v.schedule, 
                   v.employment,
                   COUNT(v.id) cnt
            FROM public.vacancies v
            GROUP BY v.schedule, v.employment
            ORDER BY cnt DESC
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

display(df)

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [58]:
fig = px.sunburst(df, path=['employment', 'schedule'], values = 'cnt',
                  width=750, height=750)
fig.update_layout(margin = dict(t=50, l=100, r=0, b=0), title='Тип занятости и график работы') \
    .update_traces(texttemplate="%{label}<br>%{percentEntry:.2%}")
fig.show();
fig.write_image('images/gr1.svg')

![](images/gr1.svg)

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [59]:
query = f'''SELECT v.experience,
                   COUNT(v.id) count
            FROM public.vacancies v
            GROUP BY v.experience
            ORDER BY count
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

display(df.style.hide(axis='index'))

experience,count
Более 6 лет,1337
Нет опыта,7197
От 3 до 6 лет,14511
От 1 года до 3 лет,26152


In [60]:
fig = px.sunburst(df, path=['experience'], values = 'count',
                  width=550, height=550)
fig.update_layout(margin = dict(t=50, l=130, r=0, b=0), title='Требуемый опыт работы, указанный в вакансиях') \
    .update_traces(texttemplate="%{label}<br>%{percentEntry:.2%}")
fig.show();
fig.write_image('images/gr2.svg')

![](images/gr2.svg)

***

**Выводы по анализу таблицы с вакансиями:**
   1. Подтвердился вывод предварительного анализа о том, что в регионах расположения вакансий представлены города не только России, но также города других стран. В числе городов лидеров по числу вакансий видны города Казахстана, Беларуси, Узбекистана.
   2. Более чем в половине строк таблицы вакансий не заполнена информация о зарплате.
   3. По полям с заполненными данными о зарплате, видно что в среднем работадатели готовы предложить зарплату не ниже 70 тысяч рублей и не более 110000 рублей.
   4. Абсолютное большинство вакансий предполагает тип трудоустройства — *'полная занятость'*(таких вакансий 93,5%). **Для этого типа трудоустройства**, рабочий график *'полный день'* встречается примерно в 4.5 раза чаще в вакансиях, чем рабочий график *'удалённая работа'* ( 72% и 16% ). Вакансий с *полной занятостью на удалёнке* в свою очередь в несколько раз больше чем с *гибким графиком работы и полной занятостью* и *удалённой работой с неполной занятостью*.
   5. Наибольшее количество вакансий представленных в таблице требуют опыт работы от 1 до 3 лет, или же от 3 до 6 лет( $\approx$ в 1.8 раза меньше). Не требующие опыта работы вакансии встречаются $\approx$ в 5.7 раза реже, чем вакансии вышеперечисленных групп. Вакансий требующих опыт более шести лет менее полутора тысяч. 
   Такое распределение можно объяснить, многими причинами, среди которых: 
    - сотрудник с опытом работы быстрее начнёт приносить прибыль компании, чем сотрудник без опыта; 
    - если есть конкуренция за конкретные позиции, то можно предъявлять более высокие требования к соискателям: в том числе набирать сотрудников имеющих опыт, чем без него;
    - затраты на обучение неопытного сотрудника весьма велики и не каждая компания себе это может позволить;
    - опыт выше 6 лет ближе к экспертному: не каждая работа требует столь высокой квалификации и стоит она дороже;
    - разница между категориями в 1-3 и 3-6 лет опыта может оказаться не столь существенной в плане производительности, но меньший опыт стоит дешевле, что более выгодно компаниям.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.
(Также будет выведено общее количество вакансий в этих компаниях, для наглядности.)

In [61]:
# количество выводимых строк
n = 5

query = f'''SELECT e.name employer,
                   COUNT(v.id) vacancies_total
            FROM public.vacancies v
            JOIN public.employers e ON e.id = v.employer_id 
            GROUP BY employer
            ORDER BY vacancies_total DESC
            LIMIT {n}
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df)

,employer,vacancies_total
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [62]:
# количество выводимых строк, не более
n = 10

# поскольку у Postgres(на момент выполнения работы)
# нет оптимизации COUNT(DISTINCT), пришлось усложнить запрос
query = f'''SELECT q.a_name, COUNT(q.e_id) emp_cnt, COUNT(q.id) vac_cnt
            FROM
                (SELECT *
                 FROM
                    (SELECT a.id a_id, a.name a_name, e.id e_id, e.name e_name
                     FROM public.areas a
                     LEFT JOIN public.employers e ON a.id = e.area) AS q
                 LEFT JOIN public.vacancies v 
                 ON q.a_id = v.area_id AND q.e_id = v.employer_id) AS q
            GROUP BY q.a_name
            ORDER BY vac_cnt, emp_cnt DESC
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df.head(n))

,a_name,emp_cnt,vac_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Великобритания,23,0
4,Краснодарский край,19,0
5,Эстония,19,0
6,Беларусь,18,0
7,Ростовская область,18,0
8,Азербайджан,17,0
9,Республика Татарстан,16,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [63]:
# количество выводимых строк, не более
n = 10

query = f'''SELECT e.name, COUNT(DISTINCT v.area_id) cnt
            FROM public.employers e
            JOIN public.vacancies v ON e.id = v.employer_id
            GROUP BY e.name
            ORDER BY cnt DESC
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df.head(n))

,name,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


In [64]:
print(f'У компании "Яндекс" наибольшее количество регионов размещения вакансий, таких регионов: {df.iloc[0]["cnt"]}')

У компании "Яндекс" наибольшее количество регионов размещения вакансий, таких регионов: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [65]:
query = f'''SELECT COUNT(e.id)
            FROM public.employers e
            LEFT JOIN public.employers_industries ei 
            ON e.id = ei.employer_id
            WHERE ei.industry_id IS NULL
         '''

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [66]:
# результат запроса
print(f'Количество работодателей, у которых не указана сфера деятельности: {answer[0]}')

Количество работодателей, у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [67]:
# количество выводимых строк, не более
n = 10

query = f'''SELECT e.name
            FROM public.employers e
            LEFT JOIN public.employers_industries ei 
            ON e.id = ei.employer_id
            GROUP BY e.id
            HAVING COUNT(ei.industry_id) = 4
            ORDER BY e.name
            LIMIT {n}
         '''

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df)

,name
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
5,5lb
6,5 Армия
7,741 Studios
8,AAEngineering Group
9,ADAPTER


In [68]:
print(f'Третья компания в алфавитном списке, с четырьмя сферами деятельности: {df.iloc[2]["name"]}')

Третья компания в алфавитном списке, с четырьмя сферами деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [69]:
industry_name = "%Разработка программного обеспечения%"

query = text(f'''SELECT COUNT(e.employer_id)
                 FROM public.employers_industries e
                 JOIN public.industries i
                 ON e.industry_id = i.id
                 WHERE(i.name) LIKE '{industry_name}'
            ''')

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [70]:
print(f'Количество работодателей, которые занимаются разработкой ПО: {answer[0]}')

Количество работодателей, которые занимаются разработкой ПО: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [71]:
# код для получения списка городов-милионников
# адрес страницы c городами
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%' +\
      'D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%' +\
      'D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

# сохранение списка городов вне локальной переменной
cities = []

# внимание: структура сайта может измениться
# в момент выполнения задания нужный список городов,
# находится во второй таблице
with requests.get(url) as response:
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('tbody')
    rows = tables[1].find_all('tr')
    
    for row in rows[1:]:
        cities.append(row.find('a')['title'])

In [72]:
company_name = "Яндекс"
cities_tuple = tuple(cities) 

query = text(f'''WITH company_vacancies_in_big_cities AS (
                     SELECT a.name, COUNT(v.id) cnt
                     FROM public.areas a
                     JOIN public.vacancies v ON a.id = v.area_id
                     JOIN public.employers e ON v.employer_id = e.id
                     WHERE e.name = '{company_name}' 
                         AND a.name IN {cities_tuple}
                     GROUP BY a.name
                     ORDER BY cnt
                 )
                      
                 SELECT *
                 FROM company_vacancies_in_big_cities
                 UNION ALL
                 SELECT 'Total', SUM(cnt)::int
                 FROM company_vacancies_in_big_cities
            ''')

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df.style.hide(axis='index'))

name,cnt
Омск,21
Челябинск,22
Красноярск,23
Волгоград,24
Пермь,25
Казань,25
Ростов-на-Дону,25
Уфа,26
Самара,26
Краснодар,30


***

**Выводы по анализу работодателей:**

1. Среди компаний, находящихся в исследуемой базе данных, наибольшее количество вакансий размещают IT компании, банки и компании занимающиеся добычей полезных ископаемых.
2. Видно, что не все работадатели базы разместили какие-либо вакансиии.
3. Часть крупных компаний стремятся разместить вакансии во многих регионах. Это связано как с предоставлением услуг в этих регионах, так и вероятно с желанием охватить более широкий круг людей, что увеличивает вероятность получения лучших специалистов в штат.
4. Чуть меньше, чем треть работадателей не указали сферу деятельности. Возможно в списке сфер деятельности не присутствует необходимых сфер или компания является широко известной в регионе размещения.
5. 15% компаний занимаются разработкой ПО. Можно предположить, что в ближайшее время, количество таких компаний будет расти: постепенно все сферы деятельности человека автоматизируются и роботизируются, и для управления требуются программы.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [73]:
# искомые в названиях вакансий подстроки
pat1 = '%data%'
pat2 = '%данн%'

query = text(f'''SELECT COUNT(v.name)
                 FROM public.vacancies v
                 WHERE lower(v.name) LIKE '{pat1}'
                     OR lower(v.name) LIKE '{pat2}'
            ''')

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [74]:
print(f'Количество вакансий, которые содержат заданные подстроки составляет: {answer[0]}')

Количество вакансий, которые содержат заданные подстроки составляет: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [75]:
# текст запроса
company_name = "Яндекс"
cities_tuple = tuple(cities) 

query = text(f'''WITH ds AS (
                     SELECT *
                     FROM public.vacancies v
                     WHERE (v.name ILIKE ANY(
                         ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
                         OR v.name LIKE '%ML%') 
                         AND v.name NOT ILIKE '%HTML%'
                 )
                 
                 SELECT COUNT(ds.name)
                 FROM ds
                 WHERE ds.name ILIKE '%junior%' 
                     OR ds.experience ILIKE '%нет опыта%'
                     OR ds.employment ILIKE '%стажировка%'
             ''')

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

print(f'Количество вакансий для начинающего дата-сайентиста составляет: {df.iloc[0, 0]}')

Количество вакансий для начинающего дата-сайентиста составляет: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [76]:
query = text(f'''WITH ds_vacancies AS (
                     SELECT *
                     FROM public.vacancies v
                     WHERE (v.name ILIKE ANY(
                         ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
                         OR v.name LIKE '%ML%') 
                         AND v.name NOT ILIKE '%HTML%'
                 )
                 
                 SELECT COUNT(v.name)
                 FROM ds_vacancies v
                 WHERE v.key_skills ILIKE ANY(ARRAY['%sql%', '%postgres%'])
             ''')

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [77]:
# результат запроса
print(f'Количество вакансий для исследователей данных,\
которые содержат ключевые навыки SQL и Postgres: {answer[0]}')

Количество вакансий для исследователей данных,которые содержат ключевые навыки SQL и Postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [78]:
# текст запроса
query = text(f'''WITH ds_vacancies AS (
                     SELECT *
                     FROM public.vacancies v
                     WHERE (v.name ILIKE ANY(
                         ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
                         OR v.name LIKE '%ML%') 
                         AND v.name NOT ILIKE '%HTML%'
                 )
                 
                 SELECT COUNT(v.name)
                 FROM ds_vacancies v
                 WHERE v.key_skills ILIKE '%python%'
             ''')

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [79]:
print(f'Количество вакансий для исследователей данных,\
которые требуют знания языка Python составляет {answer[0]}')

Количество вакансий для исследователей данных,которые требуют знания языка Python составляет 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [80]:
# текст запроса
query = text(f'''WITH ds_vacancies AS (
                     SELECT *
                     FROM public.vacancies v
                     WHERE (v.name ILIKE ANY(
                         ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
                         OR v.name LIKE '%ML%') 
                         AND v.name NOT ILIKE '%HTML%'
                 )
                 
                 SELECT avg(cardinality(regexp_split_to_array(ds.key_skills, '\t')))
                 FROM ds_vacancies ds
             ''')

# сюда будет записана полученная из базы информация
answer = None

with engine.connect() as connection:
     result = connection.execute(query)
     answer = result.first()

In [81]:
print(f'Усреднённое количество навыков, которые требуются дата-сайнтистам {round(answer[0],2)} штук')

Усреднённое количество навыков, которые требуются дата-сайнтистам 6.41 штук


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

**часть задания будет выполнена несколько другим запросом: через CASE*

In [82]:
query = text(f'''WITH ds_vacancies AS (
                     SELECT *
                     FROM public.vacancies v
                     WHERE (v.name ILIKE ANY(
                         ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
                         OR v.name LIKE '%ML%') 
                         AND v.name NOT ILIKE '%HTML%'
                 )
                 
                 SELECT v.experience, 
                        ROUND(AVG(
                            CASE WHEN v.salary_from IS NULL THEN v.salary_to
                                 WHEN v.salary_to IS NULL THEN v.salary_from
                                 ELSE (v.salary_from + v.salary_to) / 2
                            END
                        )) average_salary
                 FROM ds_vacancies v
                 WHERE v.salary_from IS NOT NULL
                       OR v.salary_to IS NOT NULL
                 GROUP BY v.experience
                 ORDER BY v.experience
             ''')

with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)
    
display(df)

,experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [83]:
print(f'Зарплата дата-сайентиста с опытом работы от трёх до шести лет: {df.iloc[2]["average_salary"]} у.е.')

Зарплата дата-сайентиста с опытом работы от трёх до шести лет: 243115.0 у.е.


***

**Выводы по предметному анализу:**

1. 3.5% связаны с профессией дата-сайентиста. Так как количество используемой информации в мире увеличивается, количество вакансий в этой сфере будет расти.
2. Несмотря на 1771 вакансию, которые относится к DS, лишь 51 вакансия  может быть предложена соискателям без опыта работы.
Это 3% всех вакансий в этой сфере. Таким образом, направления деятельности связанные с обучением профессии дата-сайентиста являются выгодными для создания. Так же эта цифра может сказать о том, что рост зарплаты с увелечением опыта работы будет существенным, что было показано в одном из пунктов выше.
3. Среди ключевых навыков Python указывают лишь в 20% случаев, а требования к работе с языком SQL или СУБД Postgres в 11%.
На данный момент анализа прочих требований и ключевых навыков не проводилось, но можно предугадать что в требованиях, указывают более специфичные вещи. Тогда как эти навыки являются основными и их владение подразумевеются без упоминания.
4. Величина зарплаты дата-сайентиста с опытом работы  более трёх лет близится к 250000 рублей, что более чем в два раза превосходит среднее верхнее значение зарплатной вилки из всех профессий безотносительно опыта.

# Дополнительное исследование данных.

Для более точного понимания входящих в таблицу вакансий предлагаемых компаниями будет проведено
небольшое дополнительное исследование. В исследовании ниже не буду постоянно запрашивать данные из базы. Размер интересующей таблицы небольшой, поэтому необходимые данные будут загружены в столбцы DataFrame.

In [84]:
# получение всей таблицы вакансий
query = f'SELECT * FROM public.vacancies v'
            
with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

display(df.head(5))

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740


Во-первых интересно посмотреть какие ещё направления деятельности входят в список вакансий. Кажется полезнным для общей картины выделить IT специальности, за исключением дата-сайентистов и посмотреть какую долю среди всех вакансий они занимают. Также выделить группу руководителей и управленцев.

In [85]:
# функция соотносит название вакансии и сферу деятельности, к которой она относится
def define_sphere(vacancy_name):
    name = vacancy_name.lower()
    
    ds_spec = ['data scientist', 'data science', 'исследователь данных', 'machine learning',]
    it_spec = ['разраб', 'develop', 'прогр', 'аналит', 'админ', 'тест',]
    manag_spec = ['менед', 'manag', 'руководит', 'начальн', 'тимлид', 'lead',]
    
    for w in ds_spec: 
        if (w in name)\
            or ('ML' in vacancy_name and 'html' not in name)\
            or ('машинн' in name and 'обучен' in name):
                return 1
    
    for w in it_spec:
        if w in name: 
            return 2
        
    for w in manag_spec:
        if w in name: 
            return 3
    
    return 4
    
    
# признак сферы деятельности
df['field_of_activity'] = pd.Series(map(define_sphere, df['name']))

In [86]:
df['field_of_activity'] = df['field_of_activity'].astype('category')
df['field_of_activity'] = df['field_of_activity'].cat.rename_categories({1:'data-scientists', 2:'it specialists',
                                                                        3:'managers', 4:'not it'})

fig = px.sunburst(df, path=['field_of_activity'],
                  width=550, height=550)
fig.update_layout(margin = dict(t=50, l=130, r=0, b=0), title='Сферы деятельности в вакансиях') \
    .update_traces(texttemplate="%{label}<br>%{percentEntry:.2%}")
fig.show();
fig.write_image('images/gr3.svg')

![](images/gr3.svg)

**Выводы:** хорошо видно, что даже при грубом разбиении по категориям, к IT-специальностям относится не менее 55% вакансий. И в тоже время специалистов направления data science мало относительно других it-профессий.

In [87]:
fig = px.box(df, x = 'salary_from',
             color = 'field_of_activity',
             title = 'Разброс минимальных зарплат по разным сферам деятельности')
fig.show();
fig.write_image('images/gr4.svg')

![](images/gr4.svg)

In [88]:
fig = px.box(df, x = 'salary_to', color = 'field_of_activity',
             title = 'Разброс максимальных зарплат по разным сферам деятельности')
fig.show();
fig.write_image('images/gr5.svg')

![](images/gr5.svg)

**Выводы:** связано ли это с малым количеством специалистов, либо с большой выгодой для бизнеса, хорошо заметна относительная привелигированность data scientist-ов по зарплате относительно других it-специальностей,  и вообще всего рынка вакансий в целом.

# Общий вывод по проекту

В данном проекте были составлены многочисленных SQL-запросы позволившие изучить рынок вакансий. 
Проект хорошо показывает, возможности по извлечению нужной информации хранящейся в базе данных, применяя предложение SELECT
языка запросов SQL.<br/>
В целом, если говорить про рынок вакансий, то хорошо заметно, что большая часть вакансий относится к IT сфере. Повышение цифровизации бизнесов и бизнес-процессов в свою очередь приводит, к генерации большего количества информации. Для работы с этими многочисленными данными, бизнес вынужден привлекать специалистов по работе с данными. Поэтому показанная выше привелигированность профессии data science является закономерной тенденцией ближайших лет.